In [2189]:
import numpy as np
import matplotlib.pyplot as plt

In [2190]:
from loaders import clean_text_for_pattern_letter
from temp_text import template_str
input_str = clean_text_for_pattern_letter(template_str, out_len=500)
print(len(input_str))
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
               'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', '\'', '1']
print(f'{letters=}')

len(letters)=32
500
letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', "'", '1']


In [2191]:
def predict_train(input_str, letters, max_index:int = 4,):
    next_index: dict[str, int] = {}
    index_by_prev: dict[str, dict] = {}

    StatsDict: dict[str, dict] = {}

    for letter in letters:
        next_index[letter] = 1

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_idx = inp_idx - 1
        prev_inp = input_str[prev_idx]
        
        inp_index = next_index[inp]

        prev_index = next_index[prev_inp]

        stats_key = prev_inp + str(prev_index) + ':' + inp + str(inp_index)
        StatsDict.setdefault(stats_key, {})

        next_inp = input_str[inp_idx + 1]
        StatsDict[stats_key].setdefault(next_inp, 0)

        StatsDict[stats_key][next_inp] += 1

        by_prev_key = prev_inp + str(prev_index) + ':' + inp
        index_by_prev.setdefault(by_prev_key, {})
        index_by_prev[by_prev_key].setdefault(inp_index, 0)
        index_by_prev[by_prev_key][inp_index] += 1

        by_prev_key = prev_inp + str(0) + ':' + inp
        index_by_prev.setdefault(by_prev_key, {})
        index_by_prev[by_prev_key].setdefault(inp_index, 0)
        index_by_prev[by_prev_key][inp_index] += 1
        
        next_index[prev_inp] += 1
        if next_index[prev_inp] > max_index:
            next_index[prev_inp] = 1
    
    return StatsDict, index_by_prev

In [2192]:
def predict(input_str, TF:list, StatsDict:dict, index_by_prev:dict, max_index, err = 0,):
    prev_index_l = [x for x in range(1, max_index+1)]

    output = []

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_inp = input_str[inp_idx - 1]
        
        index_l = []
        stats_d = {}
        index_d = {}
        for prev_index in prev_index_l:
            by_prev_key = prev_inp + str(prev_index) + ':' + inp
            # print(by_prev_key)
            inp_index_arr = index_by_prev.get(by_prev_key, {-1: -1})
            # print(inp_index_arr)

            for inp_index, val in inp_index_arr.items():
                if inp_index != -1:
                    index_d.setdefault(inp_index, 0)
                    index_d[inp_index] += val

                if inp_index != -1 and index_l.__contains__(inp_index) == False:
                    index_l.append(inp_index)

                    stats_key = prev_inp + str(prev_index) + ':' + inp + str(inp_index)
                    stats = StatsDict.get(stats_key, {})

                    next_inp = input_str[inp_idx + 1]
                    # print(next_inp, stats_key, stats)
                    for st_key, st in stats.items():
                        stats_d.setdefault(st_key, 0)
                        stats_d[st_key] += 1

        max_key_d = max(index_d, key=index_d.get)
        if inp == prev_inp:
            pred_max_key = max_key_d
            max_key_d += 1
            if max_key_d > max_index:
                max_key_d = 1
            
            index_d[max_key_d] = index_d[pred_max_key]
            index_d.pop(pred_max_key)

        if len(stats_d) > 0:
            max_key = max(stats_d, key=stats_d.get)
            print(max_key, next_inp, stats_d, index_l, index_d, prev_index_l, inp == prev_inp)
            # B B {'B': 1} [3] {3: 1} [3] False
            output.append(max_key)

        prev_index_l = list(index_d.keys())
        
        if max_key != next_inp:
            err += 1

        tf = len(index_l)
        TF.append(tf)
    
    output.insert(0, input_str[1])
    
    return TF, err, output

In [2193]:
# train
max_index_L1 = 4

index_by_prev_L1: dict[str, dict] = {}
StatsDict_L1: dict[str, dict] = {}

StatsDict, index_by_prev_L1 = predict_train(letters=letters, input_str=input_str, max_index=max_index_L1)

In [2194]:
for sdk, sdv in index_by_prev_L1.items():
    print(sdk, sdv)
    print('')

t1:h {1: 2}

t0:h {1: 4, 2: 2, 3: 3, 4: 2}

h1:e {1: 1, 4: 3}

h0:e {1: 1, 3: 3, 4: 5, 2: 4}

e1:  {1: 2, 4: 1}

e0:  {1: 6, 3: 4, 4: 4, 2: 3}

 1:s {1: 1}

 0:s {1: 1, 3: 2, 4: 1, 2: 2}

s1:k {1: 1}

s0:k {1: 1}

k1:y {1: 1}

k0:y {1: 1}

y1:  {2: 1}

y0:  {2: 1, 1: 2, 3: 1}

 2:a {1: 1}

 0:a {1: 3, 3: 5, 2: 2, 4: 2}

a1:b {1: 1}

a0:b {1: 1}

b1:o {1: 1}

b0:o {1: 1, 2: 2}

o1:v {1: 1}

o0:v {1: 1}

v1:e {2: 1}

v0:e {2: 1, 3: 2, 1: 1}

e2:  {3: 3, 2: 1, 1: 2}

 3:t {2: 2, 3: 1, 4: 1}

 0:t {2: 5, 4: 3, 1: 3, 3: 3}

t2:h {2: 2, 1: 2, 4: 1}

h2:e {3: 2, 2: 2}

e3:  {4: 2, 1: 1, 2: 2}

 4:p {1: 1}

 0:p {1: 2, 4: 1}

p1:o {2: 1}

p0:o {2: 1}

o2:r {1: 2}

o0:r {1: 3, 2: 2, 4: 1}

r1:t {3: 1}

r0:t {3: 1}

t3:  {1: 1}

t0:  {1: 2, 4: 1}

 1:w {1: 1}

 0:w {1: 1, 2: 2, 4: 2, 3: 2}

w1:a {2: 1}

w0:a {2: 3, 1: 2}

a2:s {2: 2, 1: 1}

a0:s {2: 2, 1: 3, 3: 3}

s2:  {2: 1, 4: 1}

s0:  {2: 2, 4: 7, 1: 2, 3: 3}

 2:t {4: 1, 2: 1, 3: 2}

t4:h {3: 2, 4: 1}

h3:e {4: 2, 2: 1}

e4:  {3: 1, 1: 1, 4

In [2195]:
for sdk, sdv in StatsDict.items():
    print(sdk, sdv)
    print('')

t1:h1 {'e': 2}

h1:e1 {' ': 1}

e1: 1 {'s': 1, 'd': 1}

 1:s1 {'k': 1}

s1:k1 {'y': 1}

k1:y1 {' ': 1}

y1: 2 {'a': 1}

 2:a1 {'b': 1}

a1:b1 {'o': 1}

b1:o1 {'v': 1}

o1:v1 {'e': 1}

v1:e2 {' ': 1}

e2: 3 {'t': 1, 's': 1, 'c': 1}

 3:t2 {'h': 2}

t2:h2 {'e': 2}

h2:e3 {' ': 2}

e3: 4 {'p': 1, 'f': 1}

 4:p1 {'o': 1}

p1:o2 {'r': 1}

o2:r1 {'t': 1, 'd': 1}

r1:t3 {' ': 1}

t3: 1 {'w': 1}

 1:w1 {'a': 1}

w1:a2 {'s': 1}

a2:s2 {' ': 1, 'e': 1}

s2: 2 {'t': 1}

 2:t4 {'h': 1}

t4:h3 {'e': 1, 'i': 1}

h3:e4 {' ': 1, 'a': 1}

e4: 3 {'c': 1}

 3:c1 {'o': 1}

c1:o3 {'l': 1}

o3:l1 {'o': 1}

l1:o4 {'r': 1}

o4:r2 {' ': 2}

r2: 4 {'o': 1}

 4:o1 {'f': 1}

o1:f1 {' ': 1}

f1: 1 {'t': 1}

 1:t1 {'e': 1}

t1:e1 {'l': 1, 's': 1}

e1:l2 {'e': 1}

l2:e2 {'v': 1}

e2:v2 {'i': 1, 'e': 1}

v2:i1 {'s': 1}

i1:s3 {'i': 1}

s3:i2 {'o': 1}

i2:o2 {'n': 1}

o2:n1 {',': 1, 'o': 1}

n1:,1 {' ': 1}

,1: 2 {'t': 1}

 2:t2 {'u': 1}

t2:u1 {'n': 1}

u1:n2 {'e': 1}

n2:e3 {'d': 1}

e3:d1 {' ': 1}

d1: 3 {'t': 1}



In [2196]:
# predict
TF = []

TF, err, out_L1 = predict(input_str=input_str, TF=TF, StatsDict=StatsDict, index_by_prev=index_by_prev_L1, max_index=max_index_L1)

print(err, err / (len(input_str) / 100) )

e e {'e': 4} [1, 2, 4, 3] {1: 4, 2: 2, 4: 2, 3: 3} [1, 2, 3, 4] False
    {' ': 4, 'a': 1, 't': 1, 'r': 1} [1, 4, 3, 2] {1: 1, 4: 5, 3: 3, 2: 4} [1, 2, 4, 3] False
s s {'s': 2, 'd': 2, 'm': 1, 'c': 1} [1, 4, 3, 2] {1: 6, 4: 4, 3: 4, 2: 3} [1, 4, 3, 2] False
p k {'k': 1, 'a': 1, 'o': 1, 'p': 2, 'h': 1} [1, 4, 3, 2] {1: 1, 4: 1, 3: 2, 2: 2} [1, 4, 3, 2] False
y y {'y': 1} [1] {1: 1} [1, 4, 3, 2] False
    {' ': 1} [1] {1: 1} [1] False
a a {'a': 1} [2] {2: 1} [1] False
b b {'b': 1} [1] {1: 1} [2] False
o o {'o': 1} [1] {1: 1} [1] False
v v {'v': 1} [1] {1: 1} [1] False
e e {'e': 1} [1] {1: 1} [1] False
    {' ': 1} [2] {2: 1} [1] False
c t {'t': 1, 's': 1, 'c': 2, 'i': 1, 'f': 1} [3, 2, 1] {3: 3, 2: 1, 1: 2} [2] False
h h {'h': 2, 'o': 1, 'e': 1} [2, 3, 4, 1] {2: 4, 3: 3, 4: 3, 1: 1} [3, 2, 1] False
e e {'e': 4, 'r': 1} [2, 1, 4, 3] {2: 2, 1: 4, 4: 2, 3: 3} [2, 3, 4, 1] False
    {' ': 4, 'r': 1, 'a': 1, 't': 1} [3, 2, 1, 4] {3: 3, 2: 4, 1: 1, 4: 5} [2, 1, 4, 3] False
s p {'p': 1, 'f': 1,

In [2197]:
def transition_freedom(input_str: str):
    transition_freedom: dict[str, dict] = {}

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0:
            continue

        prev_idx = inp_idx - 1
        prev_inp = input_str[prev_idx]

        transition_freedom.setdefault(prev_inp, {})
        transition_freedom[prev_inp].setdefault(inp, 0)

    # print(transition_freedom)

    for tf_key, tf in transition_freedom.items():
        transition_freedom[tf_key] = len(tf)
    
    # print(transition_freedom)

    return transition_freedom
        
TF_per_symbol = transition_freedom(input_str)
TF = []
for inp in input_str:
    TF.append(TF_per_symbol[inp])
print(len(TF), TF)

500 [12, 6, 15, 22, 12, 4, 5, 22, 11, 2, 16, 3, 15, 22, 12, 6, 15, 22, 4, 16, 11, 12, 22, 5, 11, 12, 22, 12, 6, 15, 22, 8, 16, 8, 16, 11, 22, 16, 4, 22, 12, 15, 8, 15, 3, 11, 12, 11, 16, 10, 2, 22, 12, 6, 10, 15, 7, 22, 12, 16, 22, 11, 22, 7, 15, 11, 7, 22, 8, 6, 11, 10, 10, 15, 8, 2, 22, 2, 11, 12, 2, 12, 22, 10, 16, 12, 22, 8, 11, 4, 15, 22, 11, 2, 5, 22, 6, 12, 11, 10, 3, 2, 2, 22, 8, 11, 12, 15, 22, 6, 15, 11, 11, 7, 22, 12, 16, 5, 15, 16, 10, 15, 22, 12, 11, 5, 2, 22, 11, 12, 22, 6, 15, 22, 12, 6, 16, 6, 8, 7, 15, 11, 15, 7, 22, 6, 11, 12, 22, 5, 11, 5, 22, 12, 6, 11, 16, 6, 3, 6, 22, 12, 6, 15, 22, 8, 11, 16, 5, 7, 22, 11, 11, 16, 6, 10, 7, 22, 12, 6, 15, 22, 7, 16, 16, 11, 22, 16, 4, 22, 12, 6, 15, 22, 8, 6, 11, 12, 2, 22, 2, 11, 12, 2, 12, 22, 8, 11, 4, 15, 22, 5, 5, 22, 2, 16, 7, 5, 2, 12, 22, 7, 15, 3, 15, 8, 16, 4, 15, 7, 22, 12, 6, 11, 12, 22, 5, 11, 12, 12, 11, 3, 15, 22, 7, 11, 6, 3, 22, 7, 15, 4, 11, 8, 11, 15, 10, 8, 5, 2, 2, 22, 11, 12, 22, 5, 11, 12, 22, 11, 22, 12, 4

In [2198]:
TF = np.array(TF, dtype=float)
# print(TF)
# TF = np.pad(TF, (1, 1), constant_values=[1,1])
# # print(TF)
# TF = np.pad(TF, (1, 0))[:-1] - TF
# TF = TF - np.pad(TF, (1, 0))[:-1] 
# # TF = np.pad(TF, (1, 0))[:-1] - TF
# # TF = TF - np.pad(TF, (0, 1))[-1:]
# # print(TF)
# TF[np.where(TF < 0)] = 0

for idx in range(1):
    TF = TF - np.pad(TF, (1, 0))[:-1]
    TF[np.where(TF < 0)] = 0
    TF[np.where(TF <= np.mean(TF[np.where(TF > 0)]))] = 0
# # TF = np.abs(TF)
# TF[np.where(TF <= np.mean(TF[np.where(TF > 0)]))] = 0
# # TF[np.where(TF <= 0.5)] = 0
# print(TF.size, TF.max(), TF.min())
# TF[np.where(TF <= np.mean(TF))] = 0

# fig, axs = plt.subplots(1, 1, figsize=(50, 5), tight_layout=True)
# # axs.plot(np.arange(TF.size), TF)
# markerline, stemline, baseline, = axs.stem(np.arange(TF.size), TF)
# plt.setp(markerline, markersize = 1)
# axs.xaxis.set_ticks(np.arange(len(input_str)) ,input_str)
# axs.xaxis.grid(True)

plt.show()

In [2199]:
upstream_inp: dict[int, list] = {}

temp_seq = []

for inp_idx, inp in enumerate(input_str):
    tf = TF[inp_idx]
    
    if tf > 0:
        upstream_inp.setdefault(inp_idx, temp_seq.copy())
        temp_seq.clear()

    temp_seq.append(inp)

for item, val in upstream_inp.items():
    print(item, val)
    
print(len(upstream_inp))

0 []
2 ['t', 'h']
7 ['e', ' ', 's', 'k', 'y']
10 [' ', 'a', 'b']
12 ['o', 'v']
16 ['e', ' ', 't', 'h']
19 ['e', ' ', 'p']
22 ['o', 'r', 't']
26 [' ', 'w', 'a', 's']
29 [' ', 't', 'h']
36 ['e', ' ', 'c', 'o', 'l', 'o', 'r']
39 [' ', 'o', 'f']
51 [' ', 't', 'e', 'l', 'e', 'v', 'i', 's', 'i', 'o', 'n', ',']
57 [' ', 't', 'u', 'n', 'e', 'd']
62 [' ', 't', 'o', ' ', 'a']
67 [' ', 'd', 'e', 'a', 'd']
76 [' ', 'c', 'h', 'a', 'n', 'n', 'e', 'l', '.']
78 [' ', '"']
81 ['i', 't', "'"]
82 ['s']
86 [' ', 'n', 'o', 't']
90 [' ', 'l', 'i', 'k']
95 ['e', ' ', 'i', "'", 'm']
103 [' ', 'u', 's', 'i', 'n', 'g', ',', '"']
110 [' ', 'c', 'a', 's', 'e', ' ', 'h']
114 ['e', 'a', 'r', 'd']
118 [' ', 's', 'o', 'm']
127 ['e', 'o', 'n', 'e', ' ', 's', 'a', 'y', ',']
130 [' ', 'a', 's']
132 [' ', 'h']
136 ['e', ' ', 's', 'h']
144 ['o', 'u', 'l', 'd', 'e', 'r', 'e', 'd']
148 [' ', 'h', 'i', 's']
152 [' ', 'w', 'a', 'y']
160 [' ', 't', 'h', 'r', 'o', 'u', 'g', 'h']
163 [' ', 't', 'h']
170 ['e', ' ', 'c', 'r', 'o',

In [2200]:
letters_dict: dict[str, int] = {}
for let_idx, letter in enumerate(letters):
    letters_dict.setdefault(letter, let_idx)

upstream_patterns = []

for upstream_letters in upstream_inp.values():
    upstream_pattern = np.zeros(len(letters))

    for letter in upstream_letters:
        pattern_idx = letters_dict.get(letter, 0)

        upstream_pattern[pattern_idx] = 1
    
    upstream_patterns.append(upstream_pattern)

upstream_patterns = np.array(upstream_patterns)

for up_ptr in upstream_patterns:
    print(up_ptr.astype(int).tolist())

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 1, 0, 0, 0, 0, 0,

In [2201]:
layer_size = 32

I = np.zeros(upstream_patterns.shape[1])
O = np.zeros(layer_size)
W = np.ones((O.size, I.size))
Trace = np.zeros((O.size, I.size))
print('W', W.shape, W.max(), W.min(), np.median(W))

k = 1
push_delta = 0.4

prec = 1e-30

epoch_cnt = 100
eps0 = 0.02

eps = 0.05
rate_drop = 0.0

AVG_O = np.zeros(O.shape)
AVG_O_CNT = np.zeros(O.shape)

samples_idx = np.arange(upstream_patterns.shape[0])

for epoch_idx in range(epoch_cnt):
    np.random.shuffle(samples_idx)

    for i_idx in samples_idx:
        I = upstream_patterns[i_idx]

        # infer
        O = np.dot(W, I)

        # train
        winner_ids = np.argsort(O)[::-1][:1+k]
        winner = winner_ids[0]
        loosers = winner_ids[1:1+k]

        # update
        W[winner] = (1 - eps) * W[winner] + eps * (I - 0.5) + Trace[winner]*eps
        W[winner] = np.clip(W[winner], -1, 1)

        # W[loosers] = (1 - eps) * W[loosers] + (eps * (I - 0.5) * -push_delta) + Trace[loosers]*eps
        # W[loosers] = np.clip(W[loosers], -1, 1)

        # update Trace
        rate_Trace = np.zeros(I.shape)
        rate_Trace[(I > 0.5)] = 0.3
        rate_Trace[(I < 0.5)] = -0.001

        dTrace = np.zeros(W.shape)
        dTrace[winner] = rate_Trace
        Trace += dTrace
        Trace = np.clip(Trace, 0, 1)

        eps -= eps * rate_drop

print('W', W.shape, W.max(), W.min(), np.median(W))

W (32, 32) 1.0 1.0 1.0
W (32, 32) 1.0 -0.4999999999999995 -0.4999500499505954


In [2202]:
SL_symbols = [x.capitalize() for x in letters][:layer_size]
print(len(SL_symbols))

SLSymbolDict = {}

SL_symbols_d = {}
for idx, val in enumerate(SL_symbols):
    SL_symbols_d.setdefault(val + ':' + str(idx), 0)

upstream_inp_keys = list(upstream_inp.keys())

upstream_cluster_by_symbol: dict[str, int] = {}
for idx, val in enumerate(SL_symbols):
    upstream_cluster_by_symbol.setdefault(val, idx)

for upstr_idx, up_ptr in enumerate(upstream_patterns):
    O = np.dot(W, up_ptr)

    winner = np.argmax(O)
    symbol = SL_symbols[winner]
    print(SL_symbols[winner])

    # print(W[upstream_cluster_by_symbol[symbol]].tolist())

    sls = SL_symbols[winner]
    sls_key = upstream_inp_keys[upstr_idx]
    SLSymbolDict.setdefault(sls_key, sls)
    
    SL_symbols_d[SL_symbols[winner] + ':' + str(winner)] += 1

print(len(SL_symbols_d))
for key, val in SL_symbols_d.items():
    print(key, val)

print(SLSymbolDict)

32
A
Q
Z
J
I
Z
U
V
M
Q
R
E
C
 
V
G
S
"
T
C
C
D
,
"
Z
'
F
C
M
K
Z
R
X
Y
Q
Q
O
G
Q
 
.
E
Q
S
"
T
C
D
N
J
G
C
 
R
B
X
,
O
"
X
M
G
M
 
G
G
M
P
D
U
Q
Z
A
G
J
'
E
.
E
C
,
N
R
R
B
Q
E
.
G
A
L
G
 
'
K
'
V
A
O
"
U
V
M
B
J
M
X
Q
T
,
P
B
F
C
M
K
32
A:0 4
B:1 4
C:2 8
D:3 3
E:4 5
F:5 2
G:6 9
H:7 0
I:8 1
J:9 4
K:10 3
L:11 1
M:12 8
N:13 2
O:14 3
P:15 2
Q:16 9
R:17 5
S:18 2
T:19 3
U:20 3
V:21 4
W:22 0
X:23 4
Y:24 1
Z:25 5
 :26 5
.:27 3
,:28 4
":29 5
':30 4
1:31 0
{0: 'A', 2: 'Q', 7: 'Z', 10: 'J', 12: 'I', 16: 'Z', 19: 'U', 22: 'V', 26: 'M', 29: 'Q', 36: 'R', 39: 'E', 51: 'C', 57: ' ', 62: 'V', 67: 'G', 76: 'S', 78: '"', 81: 'T', 82: 'C', 86: 'C', 90: 'D', 95: ',', 103: '"', 110: 'Z', 114: "'", 118: 'F', 127: 'C', 130: 'M', 132: 'K', 136: 'Z', 144: 'R', 148: 'X', 152: 'Y', 160: 'Q', 163: 'Q', 170: 'O', 177: 'G', 180: 'Q', 183: ' ', 186: '.', 189: 'E', 192: 'Q', 199: 'S', 201: '"', 204: 'T', 205: 'C', 209: 'D', 213: 'N', 215: 'J', 219: 'G', 220: 'C', 224: ' ', 228: 'R', 230: 'B', 235: 'X', 242: ',', 248

In [2203]:
sl_input = list(SLSymbolDict.values())
print(sl_input)

['A', 'Q', 'Z', 'J', 'I', 'Z', 'U', 'V', 'M', 'Q', 'R', 'E', 'C', ' ', 'V', 'G', 'S', '"', 'T', 'C', 'C', 'D', ',', '"', 'Z', "'", 'F', 'C', 'M', 'K', 'Z', 'R', 'X', 'Y', 'Q', 'Q', 'O', 'G', 'Q', ' ', '.', 'E', 'Q', 'S', '"', 'T', 'C', 'D', 'N', 'J', 'G', 'C', ' ', 'R', 'B', 'X', ',', 'O', '"', 'X', 'M', 'G', 'M', ' ', 'G', 'G', 'M', 'P', 'D', 'U', 'Q', 'Z', 'A', 'G', 'J', "'", 'E', '.', 'E', 'C', ',', 'N', 'R', 'R', 'B', 'Q', 'E', '.', 'G', 'A', 'L', 'G', ' ', "'", 'K', "'", 'V', 'A', 'O', '"', 'U', 'V', 'M', 'B', 'J', 'M', 'X', 'Q', 'T', ',', 'P', 'B', 'F', 'C', 'M', 'K']


In [2204]:
# SL train
max_index_L2 = 4

StatsDict_L2, index_by_prev_L2 = predict_train(input_str=sl_input, letters=SL_symbols, max_index=max_index_L2)

In [2205]:
for sdk, sdv in StatsDict_L2.items():
    print(sdk, sdv)
    print('')

A1:Q1 {'Z': 1}

Q1:Z1 {'J': 1}

Z1:J1 {'I': 1}

J1:I1 {'Z': 1}

I1:Z2 {'U': 1}

Z2:U1 {'V': 1}

U1:V1 {'M': 1}

V1:M1 {'Q': 1}

M1:Q2 {'R': 1}

Q2:R1 {'E': 1}

R1:E1 {'C': 1}

E1:C1 {' ': 1}

C1: 1 {'V': 1}

 1:V2 {'G': 1}

V2:G1 {'S': 1}

G1:S1 {'"': 1}

S1:"1 {'T': 1}

"1:T1 {'C': 1}

T1:C2 {'C': 1}

C2:C2 {'D': 1}

C3:D1 {',': 1}

D1:,1 {'"': 1}

,1:"2 {'Z': 1}

"2:Z3 {"'": 1}

Z3:'1 {'F': 1}

'1:F1 {'C': 1}

F1:C4 {'M': 1}

C4:M2 {'K': 1}

M2:K1 {'Z': 1}

K1:Z4 {'R': 1}

Z4:R2 {'X': 1}

R2:X1 {'Y': 1}

X1:Y1 {'Q': 1}

Y1:Q3 {'Q': 1}

Q3:Q3 {'O': 1}

Q4:O1 {'G': 1}

O1:G2 {'Q': 1}

G2:Q1 {' ': 1}

Q1: 2 {'.': 1}

 2:.1 {'E': 1}

.1:E2 {'Q': 1}

E2:Q2 {'S': 1}

Q2:S2 {'"': 1}

S2:"3 {'T': 1}

"3:T2 {'C': 1}

T2:C1 {'D': 1}

C1:D2 {'N': 1}

D2:N1 {'J': 1}

N1:J2 {'G': 1}

J2:G3 {'C': 1}

G3:C2 {' ': 1}

C2: 3 {'R': 1}

 3:R3 {'B': 1}

R3:B1 {'X': 1}

B1:X2 {',': 1}

X2:,2 {'O': 1}

,2:O2 {'"': 1}

O2:"4 {'X': 1}

"4:X3 {'M': 1}

X3:M3 {'G': 1}

M3:G4 {'M': 1}

G4:M4 {' ': 1}

M4: 4 {'

In [2206]:
for sdk, sdv in index_by_prev_L2.items():
    print(sdk, sdv)
    print('')

A1:Q {1: 1}

A0:Q {1: 1}

Q1:Z {1: 1}

Q0:Z {1: 2}

Z1:J {1: 1}

Z0:J {1: 1}

J1:I {1: 1}

J0:I {1: 1}

I1:Z {2: 1}

I0:Z {2: 1}

Z2:U {1: 1}

Z0:U {1: 1}

U1:V {1: 1}

U0:V {1: 1, 4: 1}

V1:M {1: 1}

V0:M {1: 1, 2: 1}

M1:Q {2: 1}

M0:Q {2: 1}

Q2:R {1: 1}

Q0:R {1: 1}

R1:E {1: 1}

R0:E {1: 1}

E1:C {1: 1}

E0:C {1: 1, 3: 1}

C1:  {1: 1}

C0:  {1: 1, 3: 1}

 1:V {2: 1}

 0:V {2: 1}

V2:G {1: 1}

V0:G {1: 1}

G1:S {1: 1}

G0:S {1: 1}

S1:" {1: 1}

S0:" {1: 1, 3: 1}

"1:T {1: 1}

"0:T {1: 1, 2: 1}

T1:C {2: 1}

T0:C {2: 1, 1: 1}

C2:C {2: 1}

C0:C {2: 1}

C3:D {1: 1}

C0:D {1: 1, 2: 1}

D1:, {1: 1}

D0:, {1: 1}

,1:" {2: 1}

,0:" {2: 1}

"2:Z {3: 1}

"0:Z {3: 1}

Z3:' {1: 1}

Z0:' {1: 1}

'1:F {1: 1}

'0:F {1: 1}

F1:C {4: 1}

F0:C {4: 2}

C4:M {2: 1, 4: 1}

C0:M {2: 1, 4: 1}

M2:K {1: 1}

M0:K {1: 1}

K1:Z {4: 1}

K0:Z {4: 1}

Z4:R {2: 1}

Z0:R {2: 1}

R2:X {1: 1}

R0:X {1: 1}

X1:Y {1: 1}

X0:Y {1: 1}

Y1:Q {3: 1}

Y0:Q {3: 1}

Q3:Q {3: 1}

Q0:Q {3: 1}

Q4:O {1: 1}

Q0:O {1: 1}

O1:G

In [2207]:
# predict
TF_L2 = []

TF_L2, err_L2, out_L2 = predict(input_str=sl_input, 
                                TF=TF_L2, 
                                StatsDict=StatsDict_L2, 
                                index_by_prev=index_by_prev_L2, 
                                max_index=max_index_L2)

for o in out_L2:
    print(o)

print(err_L2, err_L2 / (len(sl_input) / 100), len(sl_input), len(out_L2) )

Z Z {'Z': 1} [1] {1: 1} [1, 2, 3, 4] False
J J {'J': 1} [1] {1: 1} [1] False
I I {'I': 1} [1] {1: 1} [1] False
Z Z {'Z': 1} [1] {1: 1} [1] False
U U {'U': 1} [2] {2: 1} [1] False
V V {'V': 1} [1] {1: 1} [2] False
M M {'M': 1} [1] {1: 1} [1] False
Q Q {'Q': 1} [1] {1: 1} [1] False
R R {'R': 1} [2] {2: 1} [1] False
E E {'E': 1} [1] {1: 1} [2] False
C C {'C': 1} [1] {1: 1} [1] False
    {' ': 1} [1] {1: 1} [1] False
V V {'V': 1} [1] {1: 1} [1] False
G G {'G': 1} [2] {2: 1} [1] False
S S {'S': 1} [1] {1: 1} [2] False
" " {'"': 1} [1] {1: 1} [1] False
T T {'T': 1} [1] {1: 1} [1] False
C C {'C': 1} [1] {1: 1} [1] False
C C {'C': 1} [2] {2: 1} [1] False
D D {'D': 1} [2] {3: 1} [2] True
, , {',': 1} [1] {1: 1} [3] False
" " {'"': 1} [1] {1: 1} [1] False
Z Z {'Z': 1} [2] {2: 1} [1] False
' ' {"'": 1} [3] {3: 1} [2] False
F F {'F': 1} [1] {1: 1} [3] False
C C {'C': 1} [1] {1: 1} [1] False
M M {'M': 1} [4] {4: 1} [1] False
K K {'K': 2} [2, 4] {2: 1, 4: 1} [4] False
Z Z {'Z': 1} [1] {1: 1} [2, 4] 

In [2208]:
def ds_predict_train_without_index(input_str: str | list,
                                   #    sl_input: list,
                                   SLSymbolDict: dict,
                                   #    StatsDict_L1: dict,
                                   #    index_by_prev_L1: dict,
                                   #    max_index_L1: int,
                                   ):
    ds_symbol = '0'
    DsSymbolsStats = {}
    DsSymbolsStatsProb = {}

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        peak_symbol = SLSymbolDict.get(inp_idx, 0)  # inp_idx - 1
        if peak_symbol != 0:
            ds_symbol = peak_symbol

        DsSymbolsStats.setdefault(ds_symbol, {})
        DsSymbolsStats[ds_symbol].setdefault(inp, 0)

        DsSymbolsStats[ds_symbol][inp] += 1

    for ds_inp, inps in DsSymbolsStats.items():
        norm = sum(inps.values())

        for symbol, symbol_stats in inps.items():
            DsSymbolsStatsProb[ds_inp + ':' + symbol] = symbol_stats / norm

    return DsSymbolsStats, DsSymbolsStatsProb

In [2209]:
DsSymbolsStats, DsSymbolsStatsProb = ds_predict_train_without_index(input_str, SLSymbolDict)
print(DsSymbolsStatsProb)

{'0:h': 1.0, 'Q:e': 0.18, 'Q: ': 0.16, 'Q:s': 0.04, 'Q:k': 0.02, 'Q:y': 0.02, 'Q:c': 0.1, 'Q:o': 0.06, 'Q:l': 0.02, 'Q:r': 0.06, 'Q:t': 0.08, 'Q:h': 0.06, 'Q:w': 0.02, 'Q:d': 0.04, 'Q:a': 0.04, 'Q:.': 0.02, 'Q:u': 0.02, 'Q:b': 0.02, 'Q:f': 0.02, 'Q:i': 0.02, 'Z: ': 0.13043478260869565, 'Z:a': 0.13043478260869565, 'Z:b': 0.043478260869565216, 'Z:e': 0.17391304347826086, 'Z:p': 0.043478260869565216, 'Z:r': 0.08695652173913043, 'Z:d': 0.13043478260869565, 'Z:o': 0.08695652173913043, 'Z:u': 0.043478260869565216, 'Z:l': 0.043478260869565216, 'Z:w': 0.043478260869565216, 'Z:s': 0.043478260869565216, 'J:o': 0.18181818181818182, 'J:v': 0.09090909090909091, 'J:d': 0.09090909090909091, 'J:y': 0.09090909090909091, "J:'": 0.09090909090909091, 'J:a': 0.18181818181818182, 'J:r': 0.18181818181818182, 'J:,': 0.09090909090909091, 'I:e': 0.25, 'I: ': 0.25, 'I:t': 0.25, 'I:h': 0.25, 'U:o': 0.09090909090909091, 'U:r': 0.18181818181818182, 'U:t': 0.2727272727272727, 'U: ': 0.18181818181818182, 'U:h': 0.090

In [2210]:
# from itertools import cycle

def ds_predict(input_str, 
               StatsDict: dict, 
               index_by_prev: dict,
               upstream_cluster_by_symbol: dict,
               letters_dict: dict,
               out_L2: list,
               SLSymbolDict: dict,
               W,
               DsSymbolsStats: dict,
               max_index):
    prev_index_l = [x for x in range(1, max_index+1)]

    err = 0
    TF = []
    output = []

    ds_out_idx = 0
    ds_inp = '0'
    # ds_pattern = np.ones(W.shape[1])

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_inp = input_str[inp_idx - 1]
        
        index_l = []
        stats_d = {}
        index_d = {}
        for prev_index in prev_index_l:
            by_prev_key = prev_inp + str(prev_index) + ':' + inp
            # print(by_prev_key)
            inp_index_arr = index_by_prev.get(by_prev_key, {-1: -1})
            # print(inp_index_arr)

            for inp_index, val in inp_index_arr.items():
                if inp_index != -1:
                    index_d.setdefault(inp_index, 0)
                    index_d[inp_index] += val

                if inp_index != -1 and index_l.__contains__(inp_index) == False:
                    index_l.append(inp_index)

                    stats_key = prev_inp + str(prev_index) + ':' + inp + str(inp_index)
                    stats = StatsDict.get(stats_key, {})

                    next_inp = input_str[inp_idx + 1]
                    # print(next_inp, stats_key, stats)
                    for st_key, st in stats.items():
                        stats_d.setdefault(st_key, 0)
                        stats_d[st_key] += 1

        # downstream
        is_peak = SLSymbolDict.get(inp_idx, 0)
        if is_peak != 0:
            if ds_out_idx == len(out_L2):
                ds_inp = '0'
                # ds_pattern = np.ones(W.shape[1])
            else:
                ds_inp = out_L2[ds_out_idx]
                # ds_pattern = W[upstream_cluster_by_symbol.get(ds_inp)]
                # ds_pattern = np.copy(W[upstream_cluster_by_symbol.get(ds_inp)])
                # ds_pattern[np.where(ds_pattern < 0)] = 0

                ds_out_idx += 1
        print(stats_d)

        # prob stats
        # norm = sum(stats_d.values())
        # for st_key, st in stats_d.items():
        #     stats_d[st_key] = st / norm

        for st_key, st in stats_d.items():
            # letter_idx = letters_dict.get(st_key)
            
            # ds_stats = ds_pattern[letter_idx]
            ds_key = ds_inp + ':' + st_key
            ds_stats = DsSymbolsStats.get(ds_key, 0)
            print('st_ds:', st_key, st, ds_key, ds_stats)
            if ds_stats <= 0:
                stats_d[st_key] = 0

                continue
            
            stats_d[st_key] *= ds_stats
            print('upd:', ds_stats)
        
        print(stats_d)
        print(input_str[inp_idx + 1], max(stats_d, key=stats_d.get))
        print()

        max_key_d = max(index_d, key=index_d.get)
        if inp == prev_inp:
            pred_max_key = max_key_d
            max_key_d += 1
            if max_key_d > max_index:
                max_key_d = 1
            
            index_d[max_key_d] = index_d[pred_max_key]
            index_d.pop(pred_max_key)

        if len(stats_d) > 0:
            max_key = max(stats_d, key=stats_d.get)
            # print(max_key, next_inp, stats_d, index_l, index_d, prev_index_l, inp == prev_inp)
            output.append(max_key)

        prev_index_l = list(index_d.keys())
        
        if max_key != next_inp:
            err += 1

        tf = len(index_l)
        TF.append(tf)
    
    output.insert(0, input_str[1])
    
    return TF, err, output

In [2211]:
DsSymbolsStats, DsSymbolsStatsProb = ds_predict_train_without_index(input_str, SLSymbolDict)

dsp_TF, dsp_err, dsp_output = ds_predict(input_str=input_str,
                                         StatsDict=StatsDict,
                                         index_by_prev=index_by_prev_L1,
                                         upstream_cluster_by_symbol=upstream_cluster_by_symbol,
                                         letters_dict=letters_dict,
                                         out_L2=out_L2,
                                         SLSymbolDict=SLSymbolDict,
                                         W=W,
                                         DsSymbolsStats=DsSymbolsStatsProb,
                                         max_index=max_index_L1)

print(dsp_err, dsp_err / (len(input_str) / 100))


{'e': 4}
st_ds: e 4 0:e 0
{'e': 0}
e e

{' ': 4, 'a': 1, 't': 1, 'r': 1}
st_ds:   4 Q:  0.16
upd: 0.16
st_ds: a 1 Q:a 0.04
upd: 0.04
st_ds: t 1 Q:t 0.08
upd: 0.08
st_ds: r 1 Q:r 0.06
upd: 0.06
{' ': 0.64, 'a': 0.04, 't': 0.08, 'r': 0.06}
   

{'s': 2, 'd': 2, 'm': 1, 'c': 1}
st_ds: s 2 Q:s 0.04
upd: 0.04
st_ds: d 2 Q:d 0.04
upd: 0.04
st_ds: m 1 Q:m 0
st_ds: c 1 Q:c 0.1
upd: 0.1
{'s': 0.08, 'd': 0.08, 'm': 0, 'c': 0.1}
s c

{'k': 1, 'a': 1, 'o': 1, 'p': 2, 'h': 1}
st_ds: k 1 Q:k 0.02
upd: 0.02
st_ds: a 1 Q:a 0.04
upd: 0.04
st_ds: o 1 Q:o 0.06
upd: 0.06
st_ds: p 2 Q:p 0
st_ds: h 1 Q:h 0.06
upd: 0.06
{'k': 0.02, 'a': 0.04, 'o': 0.06, 'p': 0, 'h': 0.06}
k o

{'y': 1}
st_ds: y 1 Q:y 0.02
upd: 0.02
{'y': 0.02}
y y

{' ': 1}
st_ds:   1 Q:  0.16
upd: 0.16
{' ': 0.16}
   

{'a': 1}
st_ds: a 1 Z:a 0.13043478260869565
upd: 0.13043478260869565
{'a': 0.13043478260869565}
a a

{'b': 1}
st_ds: b 1 Z:b 0.043478260869565216
upd: 0.043478260869565216
{'b': 0.043478260869565216}
b b

{'o': 1}
st_ds: o 1 

In [2212]:
ds_predict_train_without_index(input_str, SLSymbolDict)

({'0': {'h': 1},
  'Q': {'e': 9,
   ' ': 8,
   's': 2,
   'k': 1,
   'y': 1,
   'c': 5,
   'o': 3,
   'l': 1,
   'r': 3,
   't': 4,
   'h': 3,
   'w': 1,
   'd': 2,
   'a': 2,
   '.': 1,
   'u': 1,
   'b': 1,
   'f': 1,
   'i': 1},
  'Z': {' ': 3,
   'a': 3,
   'b': 1,
   'e': 4,
   'p': 1,
   'r': 2,
   'd': 3,
   'o': 2,
   'u': 1,
   'l': 1,
   'w': 1,
   's': 1},
  'J': {'o': 2, 'v': 1, 'd': 1, 'y': 1, "'": 1, 'a': 2, 'r': 2, ',': 1},
  'I': {'e': 1, ' ': 1, 't': 1, 'h': 1},
  'U': {'o': 1, 'r': 2, 't': 3, ' ': 2, 'h': 1, 'a': 1, 'z': 1},
  'V': {' ': 4, 'w': 3, 'a': 3, 's': 2, 'd': 2, 'e': 1, 'o': 1},
  'M': {' ': 8,
   't': 2,
   'h': 4,
   'a': 1,
   'v': 1,
   'j': 1,
   'e': 1,
   'n': 2,
   'd': 1,
   'i': 2,
   'g': 1,
   's': 1},
  'R': {' ': 4,
   'o': 2,
   'f': 1,
   'h': 1,
   'i': 2,
   's': 1,
   'e': 1,
   'd': 3,
   'c': 1,
   'u': 1,
   'l': 1,
   'r': 1,
   'n': 1,
   'k': 1},
  'E': {' ': 2,
   't': 2,
   'e': 3,
   'l': 2,
   'v': 1,
   'i': 3,
   's': 3,
   'o'

In [2213]:
def downstream_predict_train(input_str: str | list,
                             sl_input: list,
                             SLSymbolDict: dict,
                            #  ds_out: list,
                             StatsDict_L1: dict,
                            #  StatsDict_L2,
                            #  TF_L1,
                             index_by_prev_L1: dict,
                            #  index_by_prev_L2,
                             max_index_L1: int,
                            #  max_index_L2,
                             ):
    next_index: dict[str, int] = {}

    for letter in letters:
        next_index[letter] = 1

    ds_inp = '0'
    sl_inp_idx = 1

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_idx = inp_idx - 1
        prev_inp = input_str[prev_idx]

        inp_index = next_index[inp]

        prev_index = next_index[prev_inp]

        is_peak = SLSymbolDict.get(inp_idx, 0)
        if is_peak != 0:
            if sl_inp_idx == len(sl_input):
                ds_inp = '0'
            else:
                ds_inp = sl_input[sl_inp_idx]
                sl_inp_idx += 1
        # print(f'{ds_inp=}')
        stats_key = ds_inp + ':' + prev_inp + str(prev_index) + ':' + inp + str(inp_index)
        StatsDict_L1.setdefault(stats_key, {})

        next_inp = input_str[inp_idx + 1]
        StatsDict_L1[stats_key].setdefault(next_inp, 0)

        StatsDict_L1[stats_key][next_inp] += 1

        by_prev_key = ds_inp + ':' + prev_inp + str(prev_index) + ':' + inp
        index_by_prev_L1.setdefault(by_prev_key, {})
        index_by_prev_L1[by_prev_key].setdefault(inp_index, 0)
        index_by_prev_L1[by_prev_key][inp_index] += 1

        by_prev_key = ds_inp + ':' + prev_inp + str(0) + ':' + inp
        index_by_prev_L1.setdefault(by_prev_key, {})
        index_by_prev_L1[by_prev_key].setdefault(inp_index, 0)
        index_by_prev_L1[by_prev_key][inp_index] += 1

        next_index[prev_inp] += 1
        if next_index[prev_inp] > max_index_L1:
            next_index[prev_inp] = 1
        
    return StatsDict_L1, index_by_prev_L1


In [2214]:
StatsDict_L1_ds = {}
index_by_prev_L1_ds = {}

StatsDict_L1_ds, index_by_prev_L1_ds = downstream_predict_train(input_str=input_str,
                                                                sl_input=sl_input,
                                                                SLSymbolDict=SLSymbolDict,
                                                                StatsDict_L1=StatsDict_L1_ds,
                                                                index_by_prev_L1=index_by_prev_L1_ds,
                                                                max_index_L1=max_index_L1, )

for key, val in StatsDict_L1_ds.items():
    print(key, val)

0:t1:h1 {'e': 1}
Q:h1:e1 {' ': 1}
Q:e1: 1 {'s': 1}
Q: 1:s1 {'k': 1}
Q:s1:k1 {'y': 1}
Q:k1:y1 {' ': 1}
Z:y1: 2 {'a': 1}
Z: 2:a1 {'b': 1}
Z:a1:b1 {'o': 1}
J:b1:o1 {'v': 1}
J:o1:v1 {'e': 1}
I:v1:e2 {' ': 1}
I:e2: 3 {'t': 1}
I: 3:t2 {'h': 1}
I:t2:h2 {'e': 1}
Z:h2:e3 {' ': 1}
Z:e3: 4 {'p': 1}
Z: 4:p1 {'o': 1}
U:p1:o2 {'r': 1}
U:o2:r1 {'t': 1}
U:r1:t3 {' ': 1}
V:t3: 1 {'w': 1}
V: 1:w1 {'a': 1}
V:w1:a2 {'s': 1}
V:a2:s2 {' ': 1}
M:s2: 2 {'t': 1}
M: 2:t4 {'h': 1}
M:t4:h3 {'e': 1}
Q:h3:e4 {' ': 1}
Q:e4: 3 {'c': 1}
Q: 3:c1 {'o': 1}
Q:c1:o3 {'l': 1}
Q:o3:l1 {'o': 1}
Q:l1:o4 {'r': 1}
Q:o4:r2 {' ': 1}
R:r2: 4 {'o': 1}
R: 4:o1 {'f': 1}
R:o1:f1 {' ': 1}
E:f1: 1 {'t': 1}
E: 1:t1 {'e': 1}
E:t1:e1 {'l': 1}
E:e1:l2 {'e': 1}
E:l2:e2 {'v': 1}
E:e2:v2 {'i': 1}
E:v2:i1 {'s': 1}
E:i1:s3 {'i': 1}
E:s3:i2 {'o': 1}
E:i2:o2 {'n': 1}
E:o2:n1 {',': 1}
E:n1:,1 {' ': 1}
C:,1: 2 {'t': 1}
C: 2:t2 {'u': 1}
C:t2:u1 {'n': 1}
C:u1:n2 {'e': 1}
C:n2:e3 {'d': 1}
C:e3:d1 {' ': 1}
 :d1: 3 {'t': 1}
 : 3:t3 {'o': 1}
 :t3:o3 {' ': 

In [2215]:
from itertools import cycle

print(out_L2)
def downstream_predict(input_str: str | list,
                       SLSymbolDict: dict,
                       StatsDict_L1_ds: dict,
                       index_by_prev_L1_ds: dict,
                       out_L2: list,
                       max_index_L1: int,
                       TF: list,
                       err=0,):
    prev_index_l = [x for x in range(1, max_index_L1+1)]

    ds_inp = '0'
    ds_out_idx = 0
    ds_predict_out = cycle(out_L2)

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_inp = input_str[inp_idx - 1]

        # print(inp, SLSymbolDict.get(inp_idx, 0))
        is_peak = SLSymbolDict.get(inp_idx, 0)
        if is_peak != 0:
            if ds_out_idx == len(out_L2):
                ds_inp = '0'
            else:
                ds_inp = out_L2[ds_out_idx]
                ds_out_idx += 1
        print(ds_inp, is_peak)
        index_l = []
        stats_d = {}
        index_d = {}
        for prev_index in prev_index_l:
            by_prev_key = ds_inp + ':' + prev_inp + str(prev_index) + ':' + inp
            # print(by_prev_key)
            inp_index_arr = index_by_prev_L1_ds.get(by_prev_key, {-1: -1})
            # print(inp_index_arr)

            for inp_index, val in inp_index_arr.items():
                if inp_index != -1:
                    index_d.setdefault(inp_index, 0)
                    index_d[inp_index] += val

                if inp_index != -1 and index_l.__contains__(inp_index) == False:
                    index_l.append(inp_index)

                    stats_key = ds_inp + ':' + prev_inp + \
                        str(prev_index) + ':' + inp + str(inp_index)
                    stats = StatsDict_L1_ds.get(stats_key, {})

                    next_inp = input_str[inp_idx + 1]
                    # print(next_inp, stats_key, stats)
                    for st_key, st in stats.items():
                        stats_d.setdefault(st_key, 0)
                        stats_d[st_key] += 1

        max_key_d = max(index_d, key=index_d.get)
        if inp == prev_inp:
            pred_max_key = max_key_d
            max_key_d += 1
            if max_key_d > max_index_L1:
                max_key_d = 1

            index_d[max_key_d] = index_d[pred_max_key]
            index_d.pop(pred_max_key)

        if len(stats_d) > 0:
            max_key = max(stats_d, key=stats_d.get)
            # print(max_key, next_inp, stats_d, index_l, index_d, prev_index_l, inp == prev_inp)

        prev_index_l = list(index_d.keys())

        if max_key != next_inp:
            err += 1

        tf = len(index_l)
        TF.append(tf)

    return TF, err


['Q', 'Z', 'J', 'I', 'Z', 'U', 'V', 'M', 'Q', 'R', 'E', 'C', ' ', 'V', 'G', 'S', '"', 'T', 'C', 'C', 'D', ',', '"', 'Z', "'", 'F', 'C', 'M', 'K', 'Z', 'R', 'X', 'Y', 'Q', 'Q', 'O', 'G', 'Q', ' ', '.', 'E', 'Q', 'S', '"', 'T', 'C', 'D', 'N', 'J', 'G', 'C', ' ', 'R', 'B', 'X', ',', 'O', '"', 'X', 'M', 'G', 'M', ' ', 'G', 'G', 'M', 'P', 'D', 'U', 'Q', 'Z', 'A', 'G', 'J', "'", 'E', '.', 'E', 'C', ',', 'N', 'R', 'R', 'B', 'Q', 'E', '.', 'G', 'A', 'L', 'G', ' ', "'", 'K', "'", 'V', 'A', 'O', '"', 'U', 'V', 'M', 'B', 'J', 'M', 'X', 'Q', 'T', ',', 'P', 'B', 'F', 'C', 'M', 'K']


In [2216]:
TF_L1_ds = []

TF_L1_ds, err_L1_ds = downstream_predict(input_str=input_str,
                                         SLSymbolDict=SLSymbolDict,
                                         StatsDict_L1_ds=StatsDict_L1_ds,
                                         index_by_prev_L1_ds=index_by_prev_L1_ds,
                                         out_L2=out_L2,
                                         max_index_L1=max_index_L1,
                                         TF=TF_L1_ds, )

print(err_L1_ds, err_L1_ds / (len(input_str) / 100))

0 0
Q Q
Q 0
Q 0
Q 0
Q 0
Z Z
Z 0
Z 0
J J
J 0
I I
I 0
I 0
I 0
Z Z
Z 0
Z 0
U U
U 0
U 0
V V
V 0
V 0
V 0
M M
M 0
M 0
Q Q
Q 0
Q 0
Q 0
Q 0
Q 0
Q 0
R R
R 0
R 0
E E
E 0
E 0
E 0
E 0
E 0
E 0
E 0
E 0
E 0
E 0
E 0
C C
C 0
C 0
C 0
C 0
C 0
   
  0
  0
  0
  0
V V
V 0
V 0
V 0
V 0
G G
G 0
G 0
G 0
G 0
G 0
G 0
G 0
G 0
S S
S 0
" "
" 0
" 0
T T
C C
C 0
C 0
C 0
C C
C 0
C 0
C 0
D D
D 0
D 0
D 0
D 0
, ,
, 0
, 0
, 0
, 0
, 0
, 0
, 0
" "
" 0
" 0
" 0
" 0
" 0
" 0
Z Z
Z 0
Z 0
Z 0
' '
' 0
' 0
' 0
F F
F 0
F 0
F 0
F 0
F 0
F 0
F 0
F 0
C C
C 0
C 0
M M
M 0
K K
K 0
K 0
K 0
Z Z
Z 0
Z 0
Z 0
Z 0
Z 0
Z 0
Z 0
R R
R 0
R 0
R 0
X X
X 0
X 0
X 0
Y Y
Y 0
Y 0
Y 0
Y 0
Y 0
Y 0
Y 0
Q Q
Q 0
Q 0
Q Q
Q 0
Q 0
Q 0
Q 0
Q 0
Q 0
O O
O 0
O 0
O 0
O 0
O 0
O 0
G G
G 0
G 0
Q Q
Q 0
Q 0
   
  0
  0
. .
. 0
. 0
E E
E 0
E 0
Q Q
Q 0
Q 0
Q 0
Q 0
Q 0
Q 0
S S
S 0
" "
" 0
" 0
T T
C C
C 0
C 0
C 0
D D
D 0
D 0
D 0
N N
N 0
J J
J 0
J 0
J 0
G G
C C
C 0
C 0
C 0
   
  0
  0
  0
R R
R 0
B B
B 0
B 0
B 0
B 0
X X
X 0
X 0
X 0
X 0
X 0
X 0
, ,
, 0
, 0
, 0
, 0
, 0
O O
O 0
O 0
